<a href="https://colab.research.google.com/github/Frankz199/Final-Year-Project/blob/main/Vanilla_num_10_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/colab/Results

/content/drive/MyDrive/colab/Results


In [ ]:
#!pip install syft==0.2.6

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import logging
import csv

# import Pysyft to help us to simulate federated leraning
import syft as sy

# hook PyTorch to PySyft i.e. add extra functionalities to support Federated Learning
# and other private AI tools
hook = sy.TorchHook(torch)

In [ ]:
# Create clients in a dictionary so we can call upon them easier.
clients = dict()

for i in range(10):
  clients[i] = sy.VirtualWorker(hook,id=F"client_{i}")

In [ ]:
# define the args
args = {
    'use_cuda' : True,
    'batch_size' : 64,
    'test_batch_size' : 1000,
    'lr' : 0.01,
    'log_interval' : 100,
    'epochs' : 5
}

# check to use GPU or not
use_cuda = args['use_cuda'] and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# create a simple CNN net
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels = 64, kernel_size = 3, stride = 1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=64*12*12, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = F.max_pool2d(x,2)
        x = x.view(-1, 64*12*12)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate(list(clients.values())),
    batch_size=args['batch_size'], shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args['test_batch_size'], shuffle=True)

In [ ]:
# we can look at the data, it is actually pointer tensors
for images,labels in federated_train_loader:
    print(images) # batch of images pointers
    print(labels) # batch of image labels pointers
    
    print(len(images)) # len function works on pointers as well
    print(len(labels)) # we can see both are same, no of images as well as their labels
    break



(Wrapper)>[PointerTensor | me:1848597474 -> client_0:2370621865]
(Wrapper)>[PointerTensor | me:81687162738 -> client_0:76161853800]
64
64


In [ ]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()

    individual_loss = []
    individual_acc = []
    individual_weight = []
    x = 0

    with open('Vanilla-num-ten.csv', 'a', newline='') as csvfile:

      fieldnames = ['individual_loss', 'individual_acc', 'individual_weight', 'x', 'data.location.id', 'y']

      thewriter = csv.DictWriter(csvfile, fieldnames=fieldnames)

      thewriter.writeheader()

    # iterate over federated data
      for batch_idx, (data, target) in enumerate(train_loader):
          x += 1

          does_it_pass_the_decision_tree = FrancescoCode(model,data,individual_acc,individual_loss,individual_weight)

          if does_it_pass_the_decision_tree:      
              # send the model to the remote location
              model = model.send(data.location)

        
            # the same torch code that we are use to
              data, target = data.to(device), target.to(device)
            
              optimizer.zero_grad()
            
              output = model(data)
      
            # this loss is a ptr to the tensor loss 
            # at the remote location
              loss = F.nll_loss(output, target)
            
        
            # call backward() on the loss ptr,
            # that will send the command to call
            # backward on the actual loss tensor
            # present on the remote machine
              loss.backward()
              optimizer.step()
              model.get()

          if batch_idx % args['log_interval'] == 0:

              # a thing to note is the variable loss was
              # also created at remote worker, so we need to
              # explicitly get it back
              loss = loss.get()

              print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                      epoch, 
                      batch_idx * args['batch_size'], # no of images done
                      len(train_loader) * args['batch_size'], # total images left
                      100. * batch_idx / len(train_loader), 
                      loss.item()
                  )
              )
        
          thewriter.writerow({'individual_loss': individual_loss, 'individual_acc': individual_acc, 'individual_weight': individual_weight, 'x': x, 'data.location.id': data.location.id, 'y': y})

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            # add losses together
            test_loss += F.nll_loss(output, target, reduction='sum').item() 

            # get the index of the max probability class
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    return (100. * correct / len(test_loader.dataset), test_loss)

In [ ]:
def FrancescoCode(model,data,individual_acc,individual_loss,individual_weight):
  #Get the client ID of the Model that is being Sent
  print(F"Model ID: {data.location.id}")
  individual_weight.clear()
  individual_acc.clear()
  individual_loss.clear()

  latest_model = []

  # # #Iterate over the model layers to obtain the weights and biases
  # # # Here compare the weights of the previous model and the current model
  for layer in model.parameters():
    latest_model.append(layer.detach().clone())
    #print(layer)
  

  flatten_weight = torch.flatten(latest_model[6])
  average_weight = torch.mean(flatten_weight)
  individual_weight.append(average_weight)


  # Get accuracy of the model before training ( Might want to reduce the size so its faster )
  accuracy, loss = test(model,device,test_loader)

  individual_acc.append(accuracy)
  individual_loss.append(loss)
  
  return True

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['lr'])

logging.info("Starting training !!")

for epoch in range(1, args['epochs'] + 1):
        train(args, model, device, federated_train_loader, optimizer, epoch)
        test(model, device, test_loader)
